# CAMERA AND IMAGE PREPROCESSING

In [ ]:
import argparse
import cv2

ref_point = []
cropping = False
cam = cv2.VideoCapture(0)


cv2.namedWindow("test")
img_counter = 0

while True:
    ret, frame = cam.read()
    cv2.imshow("test", frame)
    if not ret:
        break
    k = cv2.waitKey(1)

    if k%256 == 27:
        # ESC pressed
        print("Escape hit, closing...")
        break
    elif k%256 == 32:
        # SPACE pressed
        img_name = "opencv_frame_{}.jpg".format(img_counter)
        cv2.imwrite(img_name, frame)
        print("{} written!".format(img_name))
        img_counter += 1

cam.release()

def shape_selection(event, x, y, flags, param):
  global ref_point, cropping
  if event == cv2.EVENT_LBUTTONDOWN:
    ref_point = [(x, y)]
    cropping = True
  elif event == cv2.EVENT_LBUTTONUP:
    ref_point.append((x, y))
    cropping = False
    cv2.rectangle(img, ref_point[0], ref_point[1], (0, 255, 0), 2)
    cv2.imshow("image", img)

img=cv2.imread(img_name)
clone = img.copy()
cv2.namedWindow("image")
cv2.setMouseCallback("image", shape_selection)

# keep looping until the 'q' key is pressed
while True:
    cv2.imshow("image", img)
    key = cv2.waitKey(1) & 0xFF
  # if the 'r' key is pressed, reset the cropping region
    if key == ord("r"):
        img = clone.copy()
  # if the 'c' key is pressed, break from the loop
    elif key == ord("c"):
        break
if len(ref_point) == 2:
    crop_img = clone[ref_point[0][1]:ref_point[1][1], ref_point[0][0]:ref_point[1][0]]
    cv2.imshow("crop_img", crop_img)
    cv2.waitKey(0)
    crop_img=cv2.resize(crop_img,(352,170))
    cv2.imwrite('test/cropped.jpeg', crop_img)


cv2.destroyAllWindows()

# DYNAMIC

In [ ]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt

# Function to crop the image and determine acceptance or rejection based on color thresholds
def cropped(img_, x1, y1, x2, y2, x3, y3, x4, y4):
    crop_img = img_[y1:y2, x1:x2]
    crop_img1 = img_[y3:y4, x3:x4]
    b, g, r, _ = np.uint8(cv2.mean(crop_img))
    b1, g1, r1, _ = np.uint8(cv2.mean(crop_img1))
    if (b < 100 and g < 100 and r < 100):
        plt.title("Overfilled-Rejected", fontsize=100)
        imgplot = plt.imshow(img_)
    elif (b1 > 100 and g1 > 100 and r1 > 100):
        plt.title("Underfilled-Rejected", fontsize=100)
        imgplot = plt.imshow(img_)
    else:
        plt.title("Accepted", fontsize=100)
        imgplot = plt.imshow(img_)

# Folder containing the images to process
folder = "test/"
images = os.listdir(folder)
i = 0
sum_x1_ = 0
sum_y1_ = 0
sum_x2_ = 0
sum_y2_ = 0
n = 0

# Plotting the images and processing them
plt.figure(figsize=[70, 50])
for image in images:
    plt.subplot(2, 3, i + 1)
    img = cv2.imread(folder + image)
    RGB_img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    edges1 = cv2.Canny(gray, 150, 300, apertureSize=5)
    lines = cv2.HoughLines(edges1, 1, np.pi/180, 50)

    # Processing each line detected in the image
    for r, theta in lines[0]:
        a = np.cos(theta)
        b = np.sin(theta)
        x0 = a * r
        y0 = b * r
        x1 = int(x0 + 1000 * (-b))
        y1 = int(y0 + 1000 * (a))
        x2 = int(x0 - 1000 * (-b))
        y2 = int(y0 - 1000 * (a))
        sum_x1_ = x1 + sum_x1_
        sum_y1_ = y1 + sum_y1_
        sum_x2_ = x2 + sum_x2_
        sum_y2_ = y2 + sum_y2_
        n = n + 1
        x1_, y1_ = 145, int(sum_y1_ / n) - 15
        x2_, y2_ = 220, int(sum_y2_ / n) - 15
        x3_, y3_ = 145, int(sum_y1_ / n) + 15
        x4_, y4_ = 220, int(sum_y2_ / n) + 15
        cropped(RGB_img, x1_, y1_, x2_, y2_, x3_, y3_, x4_, y4_)
    i += 1

# Display the plotted images
plt.show()


# STATIC

In [ ]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt

def cropped(img,x1,y1,x2,y2,x3,y3,x4,y4):
    crop_img = img[y1:y2,x1:x2]
    crop_img1 = img[y3:y4,x3:x4]
    b,g,r,_=np.uint8(cv2.mean(crop_img))
    b1,g1,r1,_=np.uint8(cv2.mean(crop_img1))
    if (b<100 and g<100 and r<100):
        plt.title("Overfilled-Rejected")
        imgplot=plt.imshow(img)
    elif(b1>100 and g1>100 and r1>100):
        plt.title("Underfilled-Rejected")
        imgplot=plt.imshow(img)
    else:
        plt.title("Accepted")
        imgplot = plt.imshow(img)


folder = "cokes/"
images = os.listdir(folder)
i = 0
sum_x1_=0
sum_y1_=0
sum_x2_=0
sum_y2_=0

n=0
plt.figure(figsize=[70,50]) # set image size
plt.subplots_adjust(wspace = 1.0)# set distance between the subplots
for image in images:
    plt.subplot(4,10,i+1)
    img = cv2.imread(folder+image)
    RGB_img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    edges1 = cv2.Canny(gray,150,300,apertureSize = 5)
    lines = cv2.HoughLines(edges1,1,np.pi/180, 50)
    for r,theta in lines[0]:
        a = np.cos(theta)
        b = np.sin(theta)
        x0 = a*r
        y0 = b*r
        x1 = int(x0 + 1000*(-b))
        y1 = int(y0 + 1000*(a))
        x2 = int(x0 - 1000*(-b))
        y2 = int(y0 - 1000*(a))
        sum_x1_=x1+sum_x1_
        sum_y1_=y1+sum_y1_
        sum_x2_=x2+sum_x2_
        sum_y2_=y2+sum_y2_
        n=n+1
        cv2.line(RGB_img,(int(sum_x1_/n),int(sum_y1_/n)), (int(sum_x2_/n),int(sum_y2_/n)), (0,0,255),2)
        x1_,y1_=145,int(sum_y1_/n)-15
        x2_,y2_=220,int(sum_y2_/n)-15
        x3_,y3_=145,int(sum_y1_/n)+15
        x4_,y4_=220,int(sum_y2_/n)+15
        crop_img=cropped(RGB_img,x1_,y1_,x2_,y2_,x3_,y3_,x4_,y4_)
    i+=1
plt.show()